In [ ]:
import os
os.environ['SPS_HOME'] = '/home/annalena/sps_fsps'

# Load the ssp template from FSPS

In [ ]:
# NBVAL_SKIP
from rubix.spectra.ssp.factory import get_ssp_template
ssp_fsps = get_ssp_template("FSPS")

In [ ]:
age_values = ssp_fsps.age
print(age_values.shape)

metallicity_values = ssp_fsps.metallicity
print(metallicity_values.shape)

In [ ]:
index = 100
metal_index = 1

# Configure pipeline

In [ ]:
from rubix.core.pipeline import RubixPipeline
# Suppose you already have a user_config or path to config
#config = "../rubix/config/pipeline_config.yaml"
import os
config = {
    "pipeline":{"name": "calc_gradient"},
    
    "logger": {
        "log_level": "DEBUG",
        "log_file_path": None,
        "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    },
    "data": {
        "name": "IllustrisAPI",
        "args": {
            "api_key": os.environ.get("ILLUSTRIS_API_KEY"),
            "particle_type": ["stars"],
            "simulation": "TNG50-1",
            "snapshot": 99,
            "save_data_path": "data",
        },
        
        "load_galaxy_args": {
        "id": 14,
        "reuse": True,
        },
        
        "subset": {
            "use_subset": True,
            "subset_size": 2,
        },
    },
    "simulation": {
        "name": "IllustrisTNG",
        "args": {
            "path": "data/galaxy-id-14.hdf5",
        },
    
    },
    "output_path": "output",
    "output_modified":  False,

    "telescope":
        {"name": "TESTGRADIENT",
         "psf": {"name": "gaussian", "size": 5, "sigma": 0.6},
         "lsf": {"sigma": 0.5},
         "noise": {"signal_to_noise": 1,"noise_distribution": "normal"},
         },
    "cosmology":
        {"name": "PLANCK15"},
        
    "galaxy":
        {"dist_z": 0.1,
         "rotation": {"type": "edge-on"},
        },
        
    "ssp": {
        "template": {
            "name": "FSPS"
        },
    },        
}
pipe = RubixPipeline(config)
rubixdata = pipe.run()

# Set target values

In [ ]:
import jax.numpy as jnp

rubixdata.stars.age = jnp.array([age_values[index], age_values[index]])
rubixdata.stars.metallicity = jnp.array([metallicity_values[metal_index], metallicity_values[metal_index]])
rubixdata.stars.mass = jnp.array([[1.0, 1.0]])
rubixdata.stars.velocity = jnp.array([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])

In [ ]:
from rubix.core.pipeline_gradient import RubixPipeline
# Suppose you already have a user_config or path to config
#config = "../rubix/config/pipeline_config.yaml"
import os
config = {
    "pipeline":{"name": "calc_gradient"},
    
    "logger": {
        "log_level": "DEBUG",
        "log_file_path": None,
        "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    },
     "data": {
        "args": {
            "particle_type": ["stars"],
        },
    },
    
    "output_path": "output",
    "output_modified":  False,

    "telescope":
        {"name": "TESTGRADIENT",
         "psf": {"name": "gaussian", "size": 5, "sigma": 0.6},
         "lsf": {"sigma": 0.5},
         "noise": {"signal_to_noise": 1,"noise_distribution": "normal"},
         },
    "cosmology":
        {"name": "PLANCK15"},
        
    "galaxy":
        {"dist_z": 0.1,
         "rotation": {"type": "edge-on"},
        },
        
    "ssp": {
        "template": {
            "name": "FSPS"
        },
    },        
}
pipe = RubixPipeline(config)
rubixdata = pipe.run(rubixdata)

target = rubixdata

# Calculate the gradient for the grid values

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np

# Initialize the pipeline
pipe = RubixPipeline(config)

# Prepare the data structure to store results
results = []

# Iterate over the age values from 1.0 to 10.3 in 0.1 steps
for age in age_values:
    age_test = jnp.array([[age, age]])
    #print(metallicity_test)
    loss = pipe.loss_only_wrt_age(age_test, rubixdata, target)
    grad_stars_age = jax.grad(pipe.loss_only_wrt_age, argnums=0)(age_test, rubixdata, target)
    results.append((age_test, loss, grad_stars_age))

In [ ]:
# Save the results to a file
with open(f'./output/loss_grid_fsps/gradient_metals{metal_index}_agevarying{index}.txt', 'w') as f:
    for age, loss, grad in results:
        f.write(f"Metallicity: {age}, Loss: {loss}, Gradient: {grad}\n")

In [ ]:
age = [result[0] for result in results]
losses = [result[1] for result in results]
gradients = [result[2] for result in results]

age = jnp.array(age)
losses = jnp.array(losses)
gradients = jnp.array(gradients)

print(age[:, :, 0])
print(losses.shape)
print(gradients[:, :, 0])

In [ ]:
import matplotlib.pyplot as plt

age = [result[0] for result in results]
losses = [result[1] for result in results]
gradients = [result[2] for result in results]

age = jnp.array(age)
losses = jnp.array(losses)
gradients = jnp.array(gradients)

#print(losses)
#print(gradients)

# Plot the gradient versus age
plt.figure(figsize=(10, 6))
plt.plot(age[:,:, 0], gradients[:, :, 0], label='Gradient wrt age')
plt.plot(age[:, :, 0], losses, label='loss wrt age')
plt.vlines(age_values[100], -50, 50, colors='r', linestyles='dashed', label='True age')
plt.xlabel('Age')
plt.ylabel('Gradient')
plt.title('Gradient vs Age')
plt.legend()
plt.grid(True)
#plt.ylim(-1e-2,1e-2)
#plt.ylim(-60,10)
#plt.xscale('log')
#plt.savefig('./output/loss/gradient_wrt_age_grid_bc.png')
plt.show()
plt.close()

In [ ]:
import matplotlib.pyplot as plt
import jax.numpy as jnp

age = [result[0] for result in results]
losses = [result[1] for result in results]
gradients = [result[2] for result in results]

age = jnp.array(age)
losses = jnp.array(losses)
gradients = jnp.array(gradients)
print(age[:, :, 0])
print(gradients[:, :, 0])
print(losses)

# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

# Plot the gradient versus age in the first subplot with crosses at data points
ax1.plot(age[:, :, 0], gradients[:, :, 0], label='Gradient wrt age', marker='x')
ax1.vlines(age_values[50], -50, 50, colors='r', linestyles='dashed', label='True age')
ax1.set_xlabel('Age')
ax1.set_ylabel('Gradient')
ax1.set_title('Gradient vs Age')
ax1.set_ylim(-0.04, 0.04)
ax1.legend()
ax1.grid(True)

# Plot the loss versus age in the second subplot with crosses at data points
ax2.plot(age[:, :, 0], losses, label='Loss wrt age', marker='x')
ax2.vlines(age_values[50], -50, 50, colors='r', linestyles='dashed', label='True age')
ax2.set_xlabel('Age')
ax2.set_ylabel('Loss')
ax2.set_title('Loss vs Age')
ax2.set_ylim(-0.001, 0.01)
ax2.legend()
ax2.grid(True)

# Save the figure
#plt.savefig('./output/loss/gradient_and_loss_wrt_age_grid_bc3.png')
plt.show()
plt.close()